## Imports

In [1]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets
from datasets import load_dataset_builder, load_dataset

from utils import replace_text_with_whitespace

In [2]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [3]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [4]:
from templates import DatasetTemplates

In [5]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

## Config

In [9]:
# all datasets used in Burns et al. (2022) (apart from story_cloze)
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "glue:qnli",
#     "imdb",
#     "piqa",
#     "super_glue:boolq",
#     "super_glue:copa",
#     "super_glue:rte",
# ]
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "glue:qnli",
#     "imdb",
#     "super_glue:boolq",
#     "super_glue:copa",
#     "super_glue:rte",
# ]
# BURNS_DATASETS = [
#     "ag_news",
#     "amazon_polarity",
#     "dbpedia_14",
#     "imdb",
#     "super_glue:boolq",
#     "super_glue:copa",
# ]
BURNS_DATASETS = ["imdb"]

VERSION = f"v2"

# SPLIT = "train"
SPLIT = "validation"

# These numbers are chosen so that both datasets have
# approximately 10k examples in total (probably a bit less for train split)
# N_PER_DATASET = 1000 if SPLIT == "train" else 1000
# N_PER_DATASET = 1200 if SPLIT == "train" else 1375
# N_PER_DATASET = 1200 if SPLIT == "train" else 1550
N_PER_DATASET = 25000 if SPLIT == "train" else 1500
if SPLIT == "train":
    # assert N_PER_DATASET <= 2490, "N_PER_DATASET must be <= 2490"
    assert N_PER_DATASET <= 25000, "N_PER_DATASET must be <= 2490"
else:
    assert N_PER_DATASET <= 1838, "N_PER_DATASET must be <= 1838"

SEED = 42 if SPLIT == "train" else 2023
SEED

2023

## Load and inspect the datasets

In [10]:
dataset_dict = {}
for dataset_path in BURNS_DATASETS:
    print(dataset_path)

    # Parse dataset name
    dataset_name = None    
    if ":" in dataset_path:
        dataset_path, dataset_name = dataset_path.split(":")
    
    
    # Get the most validation-like split
    available_splits = load_dataset_builder(
        dataset_path, name=dataset_name
    ).info.splits.keys()
    split = "validation" if "validation" in available_splits else "test"
    split = split if SPLIT != "train" else "train"
    print(split)

    # Load the dataset
    dataset = load_dataset(
        dataset_path, name=dataset_name, split=split,
    )

    # Get a desired subset of the data
    n = N_PER_DATASET if dataset.num_rows > N_PER_DATASET else dataset.num_rows
    dataset = dataset.shuffle(seed=SEED).select(range(n))

    key = f"{dataset_path}/{dataset_name}" if dataset_name else dataset_path
    dataset_dict[key] = dataset

    print("-----------------------------------")

imdb


test
-----------------------------------


In [11]:
sum(dataset.num_rows for dataset in dataset_dict.values())

1500

In [12]:
assert all([len(Counter(dataset["label"])) > 1 for dataset in dataset_dict.values()])

In [13]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {dataset.num_rows}")

imdb: 1500


In [14]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

imdb: 2


## Get the templates

In [15]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [16]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

imdb: 13


In [17]:
# dataset_name = "ag_news"
# for idx, (template_name, template) in enumerate(dataset_template_dict[dataset_name].templates.items()):
#     print(f"{idx+1}) {dataset_name}/{template_name}: {template.metadata.choices_in_prompt}")


In [18]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     good_templates = [x for x in dataset_templates.templates.values() if x.metadata.choices_in_prompt]
#     print(f"{dataset_name}: {len(good_templates)}")

## Inspect the prompts

In [19]:
# dataset_name = "super_glue/rte"
# dataset_name = "glue/qnli"

# for template_name, template in dataset_template_dict[dataset_name].templates.items():
#     # print(template_name)
#     # print(dataset_dict[dataset_name][0])
#     q, a = template.apply(
#         dataset_dict[dataset_name][1]
#     )
#     # print(q == q.strip())
#     # print(a == a.strip())
#     # print(" ".join([q, a.strip()]))
#     print(" ".join([q, a]))
#     print(len(a))
#     print("---------------------------------")


## Form the dataset for the chosen split

In [20]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_fixed_answer_choices_list()}")

#     print("---------------------------------")


In [21]:
# for dataset_name, dataset_templates in dataset_template_dict.items():
#     print(dataset_name)
#     for template_name, template in dataset_templates.templates.items():
#         print(f"{template_name}: {template.get_answer_choices_list(dataset_dict[dataset_name][0])}")

#     print("---------------------------------")

In [22]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

imdb: 13


In [23]:
%%time

# Reproducibility
random.seed(SEED)

ALLOWED_KEYS = ["text", "label", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        # Whether the sample will be truthful or not
        is_truthful = random.choice([True, False])

        # If not truthful, sample a random incorrect label
        if not is_truthful and len(dataset.features["label"].names) > 2:
            label_mappping = dataset.features["label"] # [class 1, class 2, ...]
            all_label_ids = [label_mappping.str2int(x) for x in label_mappping.names] # [0, 1, ...]
            incorrect_label_id = all_label_ids[(1 - new_entry["label"]) % len(all_label_ids)]
            incorrect_label = template.get_fixed_answer_choices_list()[incorrect_label_id]
            
        
        # Apply the template
        if is_truthful:
            new_text = " ".join(template.apply(new_entry))
        elif not is_truthful and len(dataset.features["label"].names) > 2:
            q, a = template.apply(new_entry)
            incorrect_label = replace_text_with_whitespace(a, incorrect_label)
            new_text = " ".join([q, incorrect_label])
        else:
            # Untruthful binary case
            new_entry["label"] = 1 - new_entry["label"]
            new_text = " ".join(template.apply(new_entry))

        new_entry["text"] = new_text

        # We can now change the label to whether the sample is truthful or not
        new_entry["label"] = int(is_truthful)

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

imdb
CPU times: user 2.34 s, sys: 61.7 ms, total: 2.4 s
Wall time: 3.37 s


In [24]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['text', 'label', 'original_dataset', 'template_name'],
    num_rows: 1500
})

In [25]:
current_idx = 0
my_dataset[current_idx]

{'text': 'I am shocked and amazed to find reviews short of miserable for this horrible film. I rented this "movie" or feces, whatever you wish to call it, with several friends and after thirty minutes we had to stop watching. Just listening to the dialog left a horrible taste of sour milk in my mouth. This film was about as intelligent as an ass pimple.I hope I never see that bra-less, raggedy Anne look alike (Julianne Nicholson) again.It was like watching the most putrid pilot for a sitcom that will never make it to television, but instead of being a quick but painful 30 minutes( all I could bare)this was an excruciating 90 minutes.\n\nWhat is the sentiment expressed in this text?\n\n positive',
 'label': 0,
 'original_dataset': 'imdb',
 'template_name': 'Text Expressed Sentiment'}

In [26]:
# It's unbelievable but the fourth is better
# The following movie review expresses what
# I was going to use 'The German Scream' as a summary
# This movie is by far one of the worst B-movies
# At the very beginning, the look at a control
print(my_dataset[current_idx]["text"])

I am shocked and amazed to find reviews short of miserable for this horrible film. I rented this "movie" or feces, whatever you wish to call it, with several friends and after thirty minutes we had to stop watching. Just listening to the dialog left a horrible taste of sour milk in my mouth. This film was about as intelligent as an ass pimple.I hope I never see that bra-less, raggedy Anne look alike (Julianne Nicholson) again.It was like watching the most putrid pilot for a sitcom that will never make it to television, but instead of being a quick but painful 30 minutes( all I could bare)this was an excruciating 90 minutes.

What is the sentiment expressed in this text?

 positive


In [27]:
# Counter({0: 755, 1: 745})
# Counter({1: 750, 0: 750})
# Counter({0: 774, 1: 726}
# Counter({0: 752, 1: 748})
# Counter({0: 765, 1: 735})
Counter(my_dataset["label"]), Counter(my_dataset["original_dataset"])

(Counter({0: 764, 1: 736}), Counter({'imdb': 1500}))

In [29]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_{SPLIT}_{VERSION}.parquet")
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_imdb_{SPLIT}_{VERSION}.parquet")

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1876097

In [30]:
!ls -lah datasets | grep {VERSION} | grep imdb
# !ls -lah datasets | grep {VERSION}

drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_ppo_training_raw_v2
drwxr-xr-x  3 augustas Domain Users  25K Jul  4 23:02 burns_datasets_VINC_imdb_train_raw_v2
-rw-r--r--  1 augustas Domain Users 7.4M Jul  4 23:03 burns_datasets_VINC_imdb_train_v2.parquet
-rw-r--r--  1 augustas Domain Users 1.2M Jul  4 23:06 burns_datasets_VINC_imdb_validation_v2.parquet


In [63]:
# parquet_data_files = {
#     "train": "datasets/my_dataset_train.parquet",
#     "validation": "datasets/my_dataset_validation.parquet",
# }

# my_dataset = load_dataset("parquet", data_files=parquet_data_files)
# my_dataset